### Imports

In [ ]:
import numpy as np
import cv2, sys

print('OpenCV version - ', cv2.__version__)
print('Numpy version - ', np.version.version)
print('Python version - ', sys.version)

### Charger et afficher une image avec OpenCV

In [ ]:
img = cv2.imread("Crew_Dragon.jpg")
# ou
# img = cv2.imread("PI_TD1/Crew_Dragon.jpg", cv2.IMREAD_COLOR)  # Pour ne pas garder le canal alpha (de transparence)
# ou
img_gray = cv2.imread("Crew_Dragon.jpg", cv2.IMREAD_GRAYSCALE)


In [ ]:
cv2.imshow("Image of the Crew Dragon capsule", img)

key = cv2.waitKey(0)
if key == ord("c"):
    cv2.destroyAllWindows()
else:
    print(key)

In [ ]:
cv2.imshow("Image of the Crew Dragon capsule", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imshow("Gray-levels image of the Crew Dragon capsule", img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Afficher une image avec Matplotlib

In [ ]:
import matplotlib.pyplot as plt

# Convertir l'image vers le format lu par matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Afficher l'image avec matplotlib
plt.imshow(img_rgb)
plt.title("Matplotlib image of the Crew Dragon capsule")
# Appeler show() pour verrouiller le "plot" actuel et éviter de le mettre
# à jour au prochain affichage d'une image
plt.show()

In [ ]:
plt.imshow(img_gray, cmap='gray')
plt.title("Matplotlib gray-levels image of the Crew Dragon capsule")
plt.show()

### Couper et redimensionner une image

In [ ]:
# Lire les dimensions d'une image
print("Taille de img :", img.shape)  # (height, width, channels)
print("Taille de gray :", img_gray.shape)  # (height, width)

# Obtenir une ou plusieurs dimensions d'une image
h = img.shape[0]
w = img.shape[1]
# ou
(h, w, c) = img.shape
print("h :", h)
print("w :", w)
print("c :", c)

In [ ]:
# Récupérer les lignes 90 à 540 de l'image
img_cropped_h = img_rgb[90:540]

plt.imshow(img_cropped_h)
plt.show()

In [ ]:
# Récupérer les colonnes 180 à 840 de l'image.
# Attention, on précise avec ":" que l'on récupère toutes les lignes de l'image
img_cropped = img_cropped_h[:, 180:840]

plt.imshow(img_cropped)
plt.show()

In [ ]:
(crop_h, crop_w, _) = img_cropped.shape

resized_h = crop_h//2
resized_w = crop_w//3

img_resized = cv2.resize(img_cropped, (resized_w, resized_h), interpolation=cv2.INTER_AREA)

plt.imshow(img_resized)
plt.show()

### Appliquer une rotation sur une image

In [ ]:
rotation_angle = 77  # Rotation de l'image de 77° en sens anti-horaire
rotation_matrix = cv2.getRotationMatrix2D(center=(resized_w/2, resized_h/2), angle=rotation_angle, scale=1)
img_rotated = cv2.warpAffine(img_resized, rotation_matrix, (resized_w, resized_h))

plt.imshow(img_rotated)
plt.show()

### Appliquer une translation sur une image

In [ ]:
translation_w = -50  # Translation de 50 pixels sur la gauche
translation_h = 0  # Translation de 0 pixels verticalement
translation_matrix = np.float32([[1, 0, translation_w],
                                 [0, 1, translation_h]])
img_capsule = cv2.warpAffine(img_rotated, translation_matrix, (resized_w, resized_h))

plt.imshow(img_capsule, cmap='gray')
plt.title("Capsule"),
plt.show()

### Enregistrer une image

In [ ]:
capsule_bgr = cv2.cvtColor(img_capsule, cv2.COLOR_RGB2BGR)
cv2.imwrite("capsule.jpg", capsule_bgr)

### Trouver les bords dans une image

In [ ]:
gray_capsule = cv2.cvtColor(img_capsule, cv2.COLOR_RGB2GRAY)

# Convertir l'image en int16 pour éviter les dépassements mémoire 
gray_capsule = np.array(gray_capsule, dtype=np.int16)

# On soustrait chaque pixel par celui au-dessus de lui
edge_h = np.abs(gray_capsule[1:] - gray_capsule[:-1])

plt.imshow(edge_h, cmap='gray')
plt.title("Vertical edges of the capsule")
plt.show()

In [ ]:
edge_w = np.abs(gray_capsule[:, 1:] - gray_capsule[:, :-1])
plt.imshow(edge_w, cmap='gray')
plt.title("Horizontal edges of the capsule")
plt.show()

#### Afficher dans l'image originale les bords trouvés

In [ ]:
# Vous pouvez modifier le threshold pour voir comment cela affecte l'image
threshold = 100
color_h = (255, 0, 255)  # (Rouge, Vert, Bleu)
color_w = (0, 255, 0)  # (Rouge, Vert, Bleu)
capsule_with_borders = img_capsule.copy()

for i in range(0, img_capsule.shape[0]):
    for j in range(0, img_capsule.shape[1] - 1):
        if edge_w[i,j] > threshold:
            cv2.circle(capsule_with_borders, (j, i), 1, color_w, 1)
for i in range(0, img_capsule.shape[0] - 1):
    for j in range(0, img_capsule.shape[1]):
        if edge_h[i,j] > threshold:
            cv2.circle(capsule_with_borders, (j, i), 1, color_h, 1)

plt.imshow(capsule_with_borders)
plt.title("Capsule with borders")
plt.show()

### Trouver les coins dans une image (à l'aide d'un détecteur de Harris)

In [ ]:
corners = cv2.cornerHarris(gray_capsule.astype(np.float32), blockSize=2, ksize=3, k=0.02)
# Harris : Dérivées au carré filtrées par une gaussienne
plt.imshow(corners, cmap='gray')
plt.title("Corners of the capsule")
plt.show()

In [ ]:
threshold = 0.08*np.max(np.abs(corners))
color_corner = (255, 0, 0)  # (Rouge, Vert, Bleu)
capsule_with_corners = img_capsule.copy()

for i in range(0, img_capsule.shape[0]):
    for j in range(0, img_capsule.shape[1]):
        if np.abs(corners[i,j]) > threshold:
            cv2.circle(capsule_with_corners, (j, i), 1, color_corner, 1)

plt.imshow(capsule_with_corners)
plt.title("Capsule with corners")
plt.show()

### Trouver les correspondances entre les deux images

In [ ]:
plt.imshow(img_rgb)
plt.title("Capsule docking")
plt.show()
plt.imshow(img_capsule)
plt.title("Capsule")
plt.show()

#### Trouver les keypoints et leurs descripteurs dans les deux images

In [ ]:
sift = cv2.ORB_create()

img_keypoints, img_descriptors = sift.detectAndCompute(img_rgb, None)
capsule_keypoints, capsule_descriptors = sift.detectAndCompute(img_capsule, None)

img_with_keypoints = np.copy(img_rgb)
capsule_with_keypoints = np.copy(img_capsule)

color_keypoints = (255, 0, 0)  # (Rouge, Vert, Bleu)
cv2.drawKeypoints(img_with_keypoints, img_keypoints, img_with_keypoints, color=color_keypoints)
cv2.drawKeypoints(capsule_with_keypoints, capsule_keypoints, capsule_with_keypoints, color=color_keypoints)

plt.imshow(img_with_keypoints)
plt.title("Capsule docking with keypoints")
plt.show()
plt.imshow(capsule_with_keypoints)
plt.title("Capsule with keypoints")
plt.show()

#### Trouver les keypoints correspondants dans les deux images à partir de leurs descripteurs

In [ ]:
# Trouver les correspondances entre les descripteurs
bf_matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
# Brute-force matcher : finds the closest descriptor in the second set by trying each one

matches = bf_matcher.match(img_descriptors, capsule_descriptors)
matches = sorted(matches, key=lambda x : x.distance)

#### Afficher les n meilleures correspondances

In [ ]:
n_matches = 20

matched_img = cv2.drawMatches(img_rgb, img_keypoints, img_capsule, capsule_keypoints, matches[:n_matches], None, flags=2)

plt.imshow(matched_img)
plt.title("Matches between the original and modified capsule images")
plt.show()

In [ ]:
# Affichage en plus grand
cv2.imshow("Matches between the original and modified capsule images", cv2.cvtColor(matched_img, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()